## Downloading the Dataset

You'll need to download some prerequisite Python packages in order to run all the code below. Let's install them!

In [2]:
%%capture
!pip install numpy pandas streamlit gdown pyarrow

We will download the datasets from Google Drive just like we did last week, but this time the datasets are in [Pickle](https://pythonnumericalmethods.berkeley.edu/notebooks/chapter11.03-Pickle-Files.html) and [Parquet](https://arrow.apache.org/docs/python/parquet.html) format. 

In [3]:
import os
import shutil
import gdown
import numpy as np
import pandas as pd

# Download files from Google Drive
# Based on data from: http://insideairbnb.com/get-the-data/
file_id_1 = "1m185vTdh-u7_A2ZElBvUD4SCO6oETll2"
file_id_2 = "1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX"
downloaded_file_1 = "listings_project.pkl"
downloaded_file_2 = "calendar_project.parquet"
# Download the files from Google Drive
gdown.download(id=file_id_1, output=downloaded_file_1)
gdown.download(id=file_id_2, output=downloaded_file_2)

Downloading...
From: https://drive.google.com/uc?id=1m185vTdh-u7_A2ZElBvUD4SCO6oETll2
To: /Users/dr.gauravsablok/Desktop/Read/Code/listings_project.pkl
100%|██████████| 1.42M/1.42M [00:02<00:00, 706kB/s]
Downloading...
From: https://drive.google.com/uc?id=1w41V1oWHJrBdaNJJQ4oxVBuml5CO7MQX
To: /Users/dr.gauravsablok/Desktop/Read/Code/calendar_project.parquet
100%|██████████| 1.23M/1.23M [00:04<00:00, 289kB/s]


'calendar_project.parquet'

In [ ]:
# Show all columns (instead of cascading columns in the middle)
pd.set_option("display.max_columns", None)
# Don't show numbers in scientific notation
pd.set_option("display.float_format", "{:.2f}".format)

In [4]:
# importing the dtale library 
import dtale

## Preprocessing the Dataset
#Please load the downloaded files as DataFrames (dfs). 
#The method for loading these datasets is the same as what we did on the CoRise platform.
#### Task 1: Reading the Pickle and Parquet
[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/details-dataset#corise_cl9jxhls800403b6pf9k9nh9e)
Read the Python Pickle and PyArrow Parquet files we've just downloaded as `df_list` and `df_cal`.

In [6]:
df_list = pd.read_pickle(downloaded_file_1)
df_cal = pd.read_parquet(downloaded_file_2)

Now instead of cleaning the **Calendar DataFrame**, you are going to clean the **Listings DataFrame**. You will use the same steps we used to clean the Calendar data on the CoRise platform this week. Let's first get an overview of the columns that are in this particular DataFrame 🧐.

#### Task 2: Print column names, types, and non-null values 
[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmh42v001l3b6pcxidoxp2)
Let's try and get an overview of the **Listings DataFrame**, called `df_list`. This should show us some details about the columns in the DataFrame, like the column names, their data types, and the number of non-null values.

In [ ]:
df_list.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 34 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    6165 non-null   int64  
 1   host_acceptance_rate                  5365 non-null   float64
 2   host_is_superhost                     6165 non-null   object 
 3   host_listings_count                   6165 non-null   int64  
 4   host_total_listings_count             6165 non-null   int64  
 5   neighbourhood_cleansed                6165 non-null   object 
 6   latitude                              6165 non-null   float64
 7   longitude                             6165 non-null   float64
 8   room_type                             6165 non-null   object 
 9   accommodates                          6165 non-null   int64  
 10  bedrooms                              5859 non-null   float64
 11  beds             

In [7]:
dtale.show(df_list)

2022-12-30 16:13:36,039 - WARNING  - Found pyspark version "3.2.1" installed. The pyspark version 3.2 and above has a built-in "pandas APIs on Spark" module ported from Koalas. Try `import pyspark.pandas as ps` instead. 
2022-12-30 16:13:36,041 - WARNING  - 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. Koalas will set it for you but it does not work if there is a Spark context already launched.
2022-12-30 16:13:36,605 - INFO     - Patching spark automatically. You can disable it by setting SPARK_KOALAS_AUTOPATCH=false in your environment


In [8]:
df_list.discount_per_5_days_booked.head(5)

0    5%
1    5%
2    7%
3    6%
4    9%
Name: discount_per_5_days_booked, dtype: object

In [ ]:
#### Task 3: Remove, convert, and format
[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/clean-inspect#corise_cl9wmqqgx001x3b6p54g1ka2y)
Perform this four-step process to change each of the three `discount_per_...` columns into their proper format:
1. Remove non-numeric characters, like the percent symbol, so you can perform mathematical calculations on the column
1. Change the column into a `float` data type in order to convert the data into a ratio
1. Multiply the whole column by 0.01 so you end up with a probability ratio instead of a percentage
1. Overwrite the old `discount_per_...` column with this new column
Perform these four steps for all thee columns.
***Please note that running this code block more than once might cause an error. This is because you are re-assigning your columns with this code, and if you run the code again, the variable/column you are referring to has already been changed to its preferred state.***


In [ ]:
df_list["discount_per_5_days_booked"] = (df_list["discount_per_5_days_booked"].str.replace("%", "", regex = True).astype("float")) / 100
df_list["discount_per_10_days_booked"] = (df_list["discount_per_10_days_booked"].str.replace("%", "", regex = True).astype("float")) / 100
df_list["discount_per_30_and_more_days_booked"] = (df_list["discount_per_30_and_more_days_booked"].str.replace("%", "", regex = True).astype("float")) / 100

In [9]:
df_list.discount_per_5_days_booked.head(5)

0    5%
1    5%
2    7%
3    6%
4    9%
Name: discount_per_5_days_booked, dtype: object

In [10]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,f,t,t
1,t,f,t
2,f,f,t
3,f,f,t
4,t,f,t


In [11]:
df_list["host_is_superhost"] = df_list["host_is_superhost"].replace({"t": True, "f": False}).astype("bool")
df_list["instant_bookable"] = df_list["instant_bookable"].replace({"t": True, "f": False}).astype("bool")
df_list["has_availability"] = df_list["has_availability"].replace({"t": True, "f": False}).astype("bool")

In [12]:
df_list[["host_is_superhost", "instant_bookable", "has_availability"]].head(5)

,host_is_superhost,instant_bookable,has_availability
0,False,True,True
1,True,False,True
2,False,False,True
3,False,False,True
4,True,False,True


In [13]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,$88.00,$44,$176,$4.99
1,$105.00,$52.5,$315,$4.99
2,$152.00,$38,$304,$4.99
3,$87.00,$43.5,$174,$4.99
4,$160.00,$40,$320,$4.99


In [14]:
df_list["price"] = (df_list["price"].str.replace("$", "", regex = True).str.replace(",", "", regex = True)).astype("float")
df_list["price_per_person"] = (df_list["price_per_person"].str.replace("$", "", regex = True).str.replace(",", "", regex = True)).astype("float")
df_list["minimum_price"] = (df_list["minimum_price"].str.replace("$", "", regex = True).str.replace(",", "", regex = True)).astype("float")
df_list["service_cost"] = (df_list["service_cost"].str.replace("$", "", regex = True).str.replace(",", "", regex = True)).astype("float")

In [15]:
df_list[["price", "price_per_person", "minimum_price", 'service_cost']].head(5)

,price,price_per_person,minimum_price,service_cost
0,88.0,44.0,176.0,4.99
1,105.0,52.5,315.0,4.99
2,152.0,38.0,304.0,4.99
3,87.0,43.5,174.0,4.99
4,160.0,40.0,320.0,4.99


In [18]:
df_list = df_list.rename(columns = {"price" : "price_in_dollar", "neighbourhood_cleansed" : "neighbourhood"})

In [19]:
df_list[["neighbourhood", "room_type"]] = df_list[["neighbourhood", "room_type"]].astype("category")

In [20]:
df_list = df_list.drop(columns = ["host_listings_count", "host_total_listings_count", "availability_60", "availability_90", "availability_365", "number_of_reviews", "number_of_reviews_ltm", "reviews_per_month"])

In [21]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6165 entries, 0 to 6172
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    6165 non-null   int64   
 1   host_acceptance_rate                  5365 non-null   float64 
 2   host_is_superhost                     6165 non-null   bool    
 3   neighbourhood                         6165 non-null   category
 4   latitude                              6165 non-null   float64 
 5   longitude                             6165 non-null   float64 
 6   room_type                             6165 non-null   category
 7   accommodates                          6165 non-null   int64   
 8   bedrooms                              5859 non-null   float64 
 9   beds                                  6082 non-null   float64 
 10  amenities                             6165 non-null   int64   
 11  pric

In [23]:
dtale.show(df_list)

/Users/dr.gauravsablok/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py:746: FutureWarning:

['neighbourhood', 'room_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



In [24]:
df_list["price_in_euros"].unique()

array([None], dtype=object)

In [25]:
# df_list.head(5)
df_list = df_list.drop(columns = ["price_in_euros"])

In [ ]:
df_list = df_list.dropna(axis = 0, how = "any", subset = ["review_scores_rating", "host_acceptance_rate"])

In [26]:
df_list["room_type"].unique()

['Private room', 'Entire home/apt', 'Hotel room', 'Shared room']
Categories (4, object): ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

In [27]:
def fill_empty_bedrooms(accommodates: int, bedrooms: int, room_type: str) -> int:
    if (room_type == "Private room") or (room_type == "Shared room"):
        return 1
    elif (room_type == "Hotel room") or (room_type == "Entire home/apt"):
        return np.ceil(accommodates / 2)
    else:
        return bedrooms

In [28]:
%%timeit -r 4 -n 100

temp_df = df_list.copy()  # Deep copy of the df, not a "view"
temp_df["rooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

75.7 ms ± 7.13 ms per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [ ]:
df_list["bedrooms"] = df_list[["accommodates", "bedrooms", "room_type"]].apply(
    lambda x: fill_empty_bedrooms(x["accommodates"], x["bedrooms"], x["room_type"]),
    axis=1,
)

In [29]:
%%timeit -r 4 -n 100
temp_df = df_list.copy()
temp_df['beds'] = temp_df['bedrooms']
private_shared_mask = ( 
    (temp_df['room_type'] == "Private room") | (temp_df['room_type'] == "Shared room")
)
temp_df.loc[private_shared_mask, "beds"] = 1
hotel_apt_mask = (
     (temp_df['room_type'] == "Hotel room") | (temp_df['room_type'] == "Entire home/apt")
)
temp_df.loc[hotel_apt_mask, "beds"] = np.ceil(temp_df["accommodates"] / 2)


3.21 ms ± 972 µs per loop (mean ± std. dev. of 4 runs, 100 loops each)


In [30]:
df_list = df_list.dropna(axis = 0, how = "any", subset = ["bedrooms", "beds"])

In [31]:
# df_list.info()
df_list["beds"] = df_list["beds"].astype("int")
df_list["bedrooms"] = df_list["bedrooms"].astype("int")

In [32]:
df_list["bedrooms"] = df_list["bedrooms"].astype("int8")
df_list["bedrooms"].memory_usage(index = False, deep = True)

5791

In [33]:
df_list.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5791 entries, 0 to 6172
Data columns (total 25 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   id                                    5791 non-null   int64   
 1   host_acceptance_rate                  5019 non-null   float64 
 2   host_is_superhost                     5791 non-null   bool    
 3   neighbourhood                         5791 non-null   category
 4   latitude                              5791 non-null   float64 
 5   longitude                             5791 non-null   float64 
 6   room_type                             5791 non-null   category
 7   accommodates                          5791 non-null   int64   
 8   bedrooms                              5791 non-null   int8    
 9   beds                                  5791 non-null   int64   
 10  amenities                             5791 non-null   int64   
 11  pric

In [34]:
dtale.show(df_list)

/Users/dr.gauravsablok/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py:746: FutureWarning:

['neighbourhood', 'room_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



In [35]:
# The Calendar DataFrame!
df_cal.head(3)

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights
0,23726706,2022-06-05,False,90.0,2,1125
1,23726706,2022-06-06,False,90.0,2,1125
2,23726706,2022-06-07,False,90.0,2,1125


In [42]:
# First start by making a copy, for debugging purposes
calendar_ndf = df_cal.copy()

include_list = (
    calendar_ndf["minimum_nights"] >= 3
)

In [43]:
# Get all the listings with a minimum nights of 3+
# Use the include_list
calendar_ndf = calendar_newdf.loc[include_list]

In [44]:
calendar_ndf.head(5)

,listing_id,date,available,price_in_dollar,minimum_nights,maximum_nights,five_day_dollar_price
365,35815036,2022-06-05,False,105.0,3,1125,525.0
367,35815036,2022-06-07,True,105.0,3,1125,525.0
368,35815036,2022-06-08,True,105.0,3,1125,525.0
369,35815036,2022-06-09,True,105.0,3,1125,525.0
370,35815036,2022-06-10,False,105.0,3,1125,525.0


| Related functions |
| ---- |
| [isin()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.isin.html): Filter the DataFrame on provided values |
| [eq()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.eq.html#pandas.DataFrame.eq): Filter the DataFrame for all values equal to the provided input |
| [ne()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ne.html#pandas.DataFrame.ne): Filter the DataFrame for all values not equal to the provided input |

In [45]:
calendar_ndf["five_day_dollar_price"] = calendar_newdf["price_in_dollar"] * 5 

In [46]:
calendar_summarizeddf = pd.pivot_table(
    data=calendar_ndf,
    index=["listing_id"],
    values=["available", "five_day_dollar_price"],
    aggfunc=np.mean,  # The default aggregation function used
    # for merging multiple related rows of data.
)

calendar_summarizeddf.head(3)

,available,five_day_dollar_price
listing_id,,
2818,0.208219,346.90411
44391,0.000000,1200.00000
49552,0.458101,1162.50000


#### (Extra Credit) Task 16: Maximum price and date

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/merging-pros#corise_cl9woiqve006p3b6p0vsdbqge)

Can you make a **pivot table** that states the **maximum `price_in_dollar`** for every Airbnb listing?

The expected pivot table output should look like.

|listing\_id|price\_in\_dollar|
|---|---|
|2818|80\.0|
|44391|240\.0|
|49552|300\.0|

In [47]:
temp_sum_df = pd.pivot_table(
    data = calendar_ndf, 
    index = ["listing_id"], 
    values = ["price_in_dollar"], 
    aggfunc = np.max, 
)

temp_sum_df.head(3)

,price_in_dollar
listing_id,
2818,80.0
44391,240.0
49552,300.0


#### Task 17: Mergin'

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/merging-pros#corise_cl9wp1zf9007m3b6pfiawiagd)

Let's use the [`pd.merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) operation as was shown on CoRise, with the pivot table on the right and the **Listings DataFrame** on the left. Make sure to provide which columns you want to join on for our pivot table and the DataFrame.


The expected merged table should look same as

|index|id|host\_acceptance\_rate|host\_is\_superhost|neighbourhood\_cleansed|latitude|longitude|room\_type|accommodates|bedrooms|beds|amenities|price|minimum\_nights|maximum\_nights|has\_availability|availability\_30|number\_of\_reviews\_l30d|review\_scores\_rating|instant\_bookable|price\_per\_person|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|35815036|1\.0|true|Noord-Oost|52\.42419|4\.95689|Entire home/apt|2|1|1|5|105\.0|3|100|true|4|6|4\.96|false|52\.5|
|1|19572024|1\.0|false|Watergraafsmeer|52\.30739|4\.90833|Entire home/apt|6|3|6|14|279\.0|3|300|true|6|3|4\.69|false|46\.5|
|2|2973384|0\.38|false|Watergraafsmeer|52\.30989|4\.90528|Entire home/apt|5|3|3|7|185\.0|6|21|true|0|0|4\.83|false|37\.0|

In [48]:
final_df = pd.merge(
    df_list, 
    calendar_summarizeddf, 
    left_on= "id", 
    right_on = "listing_id", 
    how = "inner"
  )
final_df.head(3)

,id,host_acceptance_rate,host_is_superhost,neighbourhood,latitude,longitude,room_type,accommodates,bedrooms,beds,...,review_scores_rating,instant_bookable,price_per_person,minimum_price,discount_per_5_days_booked,discount_per_10_days_booked,discount_per_30_and_more_days_booked,service_cost,available,five_day_dollar_price
0,19572024,1.00,False,Watergraafsmeer,52.30739,4.908330,Entire home/apt,6,3,6,...,4.69,False,46.5,837.0,9%,16%,14%,4.99,0.816438,1496.547945
1,2973384,0.38,False,Watergraafsmeer,52.30989,4.905280,Entire home/apt,5,3,3,...,4.83,False,37.0,1110.0,6%,12%,18%,4.99,0.035616,941.301370
2,34985473,1.00,True,Noord-Oost,52.42831,5.042191,Entire home/apt,5,2,4,...,4.43,False,27.0,810.0,8%,13%,17%,4.99,0.161644,748.561644


In [49]:
dtale.show(final_df)

/Users/dr.gauravsablok/opt/anaconda3/lib/python3.9/site-packages/dtale/views.py:746: FutureWarning:

['neighbourhood', 'room_type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



---

#### (Extra Credit) Task 18: Groups are great

[*\[Related section on CoRise\]*](https://corise.com/course/python-for-data-science/v2/module/data-explore-stats#corise_cl9wpp3t100n53b6pmv167sm7)

Now, let's perform a [`groupby`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html) where we look at the median values of `five_day_dollar_price` and `review_scores_accuracy` with respect to the `room_type`. Do these results match your intuition?


The expected group by table should look same as,

|room\_type|review\_scores\_rating|five\_day\_dollar\_price|
|---|---|---|
|Entire home/apt|4\.88|972\.7397260273973|
|Hotel room|4\.5649999999999995|908\.1575342465753|
|Private room|4\.79|681\.986301369863|
|Shared room|4\.6|565\.027397260274|

In [ ]:
final_df.groupby(by = ["room_type"])[["review_scores_rating", "five_day_dollar_price"]].median()

,review_scores_rating,five_day_dollar_price
room_type,,
Entire home/apt,4.88,975.00
Hotel room,4.56,1110.16
Private room,4.79,710.91
Shared room,4.60,724.11


You might have expected that shared rooms are the cheapest and thus have the lowest rating with respect to median scores. The same can't be said for the most expensive option — a hotel room. Will this influence your future considerations when booking 🤔?

(But before you let this influence your decisions too much, it might be better to assume that this data might be biased in favor of Airbnb and not hotels in general. 🤷)

---

You've walked through all the most important parts of Pandas. It's a really easy-to-use library that shares a lot of syntax with NumPy. It is great for analyzing and cleaning datasets, and as you might have discovered with the previous code, Pandas allows you to really go into the nitty-gritty details of your dataset. These skills are invaluable for a data scientist, and will empower you to utilize data where you work now, or even where you could work in the future!

The next steps involve downloading the files to your local computer so that you can make an app for your portfolio. After, we will provide some suggestions on how you can extend this project, along with some interesting links to investigate.

### Download the Dataset to Your Local Machine

Let's first export our final DataFrame.

Google Colab comes with its own Python packages that allow us to quickly download generated files, like so:

### Make an App for Your Portfolio!

<center>
  <img src=https://griddb-pro.azureedge.net/en/wp-content/uploads/2021/08/streamlit-1160x650.png width="500" align="center" />
</center>
<br/>

**Participants such as yourselves often want to use the weekly CoRise projects for their portfolios. To facilitate that, we've created this section. It might seem like a lot, but it's actually just following instructions and copy-pasting. Reach out on Slack if you get stuck!** 

You will make an app that visualizes the dataset as a DataFrame and as a geographic visualization like so:

<center>
  <img src=https://i.ibb.co/N9JKbd8/Screen-Shot-2022-11-10-at-4-07-17-PM.png width="500" align="center" />
</center>
<br/>

To visualize this, we will again use a library called [Streamlit](https://streamlit.io/). For now you are not expected to know how Streamlit works, but you are expected to be able to copy-paste and follow instructions if you want to share this project as part of your portfolio!

We are going to use [Streamlit Share](https://share.streamlit.io/) to host your projects. It's a website that allows us to host our interactive projects for free online! Again, we don't expect you to understand how to use and/or modify the code shown below. We do expect you to read the instructions and copy-paste our code to the Streamlit Share platform. Feel free to change it any way you like. Some great starting points are [here](https://python.plainenglish.io/how-to-build-web-app-using-streamlit-pandas-numpy-5e134f0cf552), [here](https://docs.streamlit.io/library/get-started/create-an-app), [here](https://streamlit.io/components), and [here](https://streamlit.io/gallery)!

The `app.py` below is based on [this code](https://github.com/tylerjrichards/st-filter-dataframe/blob/main/streamlit_app.py). Feel free to extend it if you like!

The **%%writefile [FILE_NAME].[FILE_EXTENSION]** command let's us save the code written in the cells in your Google Colab instance. Having it saved like that enables us to download it as a file, as seen below.

---

# 🎉 CONGRATULATIONS!!!

You've made it to the end of the Week 2 assignment! You should be proud. 

If you have any lingering questions, post them on Slack! As you know, we're always here to help.

And if you want any additional challenge questions, check out the bonus extensions below.

---

## Extensions (Optional)

<center>
  <img src=https://raw.githubusercontent.com/aschonfeld/dtale-media/master/images/Title.png width="500" align="center" />
</center>
<br/>

Awesome 🎉🎉🎉 you are finished with all the tasks! How about considering running [D-Tale](https://github.com/man-group/dtale) on your DataFrames, so you can interactively explore your dataset? What about getting a visual representation of the data and thus a quick overview of what's goin' on by using [Lux](https://github.com/lux-org/lux) or [Sweetviz](https://pypi.org/project/sweetviz/)?

If optimizing code gets you excited, then try to see how you can maximize these performance numbers even more. Maybe it's time to try running this code using [cuDF](https://github.com/rapidsai/cudf), [Polars](https://github.com/pola-rs/polars), or [Vaex](https://github.com/vaexio/vaex)?

# Next Up?
This is the last project of our course 😭😭. Please do not be sad! We would like to invite you to read the "What's Next?" post in Week 3, as it contains great suggestions on how you can proceed with your learning journey at CoRise (yes, please!) or somewhere else!